<a href="https://colab.research.google.com/github/iptimoshenko/openai_forbiz_tasks/blob/main/Telegram_bot_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code for chatGPT bot creation.

In [ ]:
!pip install openai
!pip install langchain
!pip install python-telegram-bot
!pip install python-telegram-bot --upgrade
!pip install faiss-cpu  # Use faiss-gpu for GPU version


In [ ]:
# Replace 'your_api_key' with your actual OpenAI API key
openai_api_key = ''
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import os
import openai
from langchain.llms import OpenAI

In [ ]:
# Function to load JSON data
def load_json(file_path):
  with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Function to save JSON data
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

In [ ]:
# Load the dialogs from the JSON file
dialogs0 = load_json('/content/drive/My Drive/ecstatic_earthling/apps/qa_prep_habit.json')
dialogs = list()
for i, item in enumerate(dialogs0):
  dialogs.append(dialogs0[item])
dialogs = [item for sublist in dialogs for item in sublist]
print(dialogs)
save_json(dialogs, 'new_dialogs.json')

[{'question': 'I’d like to address my low libido. Do you offer a body-to-body massage with mutual touch?', 'answer': 'Greetings Don, Thank you for filling in the form. I can help with awakening your libido. And my approach would be in focusing on recreating a healthy energy flow in your being rather than sourcing it from other bodies. From working with hundreds, I strongly believe that men’s focus on visual stimulation from outside is the main reason for their burnouts, low libido and similar problems. I do not offer body to body massage and my two-way touch sessions do not involve genital touch. My value is quite exceptional and something that is easily felt by clients: I have an exceptionally healthy body and strong energetic system that has been developed with 23 years of rigorous physical, mental and spiritual training. This means that my body and energetic system is capable of guiding your body to a richer state without our mind involvement. Bodies just need to be comfortable in e

In [ ]:
import openai
# Replace 'your_api_key' with your actual OpenAI API key
openai_api_key = ''
openai.api_key = openai_api_key
# Function to load JSON data
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)
# Function to save JSON data
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# Load the dialogs from the JSON file
dialogs = load_json('new_dialogs.json')

# Format the data into an FAQ structure
faq_entries = []
for entry in dialogs:
    question = entry.get("question", "Unknown Question")
    answer = entry.get("answer", "No answer provided")
    faq_entries.append({"question": question, "answer": answer})

# ... rest of your code ...

# Example of using the API to refine the first FAQ entry
response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="Refine this FAQ entry:\n\n" + faq_entries[0]["question"] + "\n" + faq_entries[0]["answer"],
  max_tokens=100
)
refined_faq_entry = response.choices[0].text.strip()

# Update the first FAQ entry with the refined version
faq_entries[0]["answer"] = refined_faq_entry

# Save the entire FAQ to a file
save_json(faq_entries, 'faq.json')

print("FAQ saved to faq.json")
for entry in faq_entries:
  print(entry)

In [ ]:
import json
import openai
import numpy as np
import faiss

# # Replace 'your_api_key' with your actual OpenAI API key
openai.api_key = ''

# Function to load JSON data
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Load the FAQ entries
faq_entries = load_json('faq.json')

# Extract texts for embeddings
texts = [entry['question'] + " " + entry['answer'] for entry in faq_entries]

# Generate embeddings using OpenAI
def get_embeddings(texts):
    return openai.Embedding.create(input=texts, engine="text-similarity-babbage-001")["data"]

embeddings = get_embeddings(texts)

# Convert embeddings to a format suitable for FAISS
embedding_matrix = np.array([embedding['embedding'] for embedding in embeddings])

# Create a FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

# Now you can use the index for similarity searches
# Example: searching for the embedding closest to the first entry
distances, indices = index.search(embedding_matrix[0:1], k=5)

print("Closest entries to the first one:", indices)

In [ ]:
# """ WORKING Code for telegram bot below """
# !pip install python-telegram-bot
# !pip install python-telegram-bot --upgrade
# from telegram import Update
# from telegram.ext import Updater, MessageHandler, Filters, CallbackContext

# # Replace 'YOUR_TOKEN' with the token you received from BotFather
# TOKEN = ''
# def reply_message(update: Update, context: CallbackContext) -> None:
#     """Reply to the user message with a specific response."""
#     context.bot.send_message(chat_id=update.effective_chat.id, text="Я не знаю, спроси у ChatGPT")

# def main() -> None:
#     """Start the bot."""
#     updater = Updater(TOKEN, use_context=True)

#     # Get the dispatcher to register handlers
#     dp = updater.dispatcher

#     # Handle all text messages
#     message_handler = MessageHandler(Filters.text & (~Filters.command), reply_message)
#     dp.add_handler(message_handler)

#     # Start the Bot
#     updater.start_polling()
#     updater.idle()

# if __name__ == '__main__':
#     main()


In [ ]:
import openai
import faiss
import numpy as np
from telegram import Update
from telegram.ext import Updater, MessageHandler, Filters, CallbackContext
# OpenAI API key
openai.api_key = ''

# Load or create your FAISS index and embeddings database
# This is a placeholder. You need to load your precomputed embeddings and FAISS index here.
embedding_matrix = embedding_matrix # np.load('your_embedding_matrix.npy')
index = index # faiss.read_index('your_faiss_index.index')

# Function to get embeddings
def get_embeddings(text):
    response = openai.Embedding.create(input=[text], engine="text-similarity-babbage-001")
    return np.array(response['data'][0]['embedding'])

# Function to find the most similar entry in the database
def find_most_similar(embedding):
    distances, indices = index.search(embedding.reshape(1, -1), k=1)
    return indices[0][0]  # Return the index of the most similar entry

# Replace 'YOUR_TOKEN' with the token you received from BotFather
TOKEN = ''

def reply_message(update: Update, context: CallbackContext) -> None:
    """Reply to the user message with a specific response."""
    # Check if the update contains a message and the message contains text
    if update.message and update.message.text:
        user_message = update.message.text
        # Process the message (e.g., find a similar embedding, respond accordingly)
        # For now, just sending a placeholder response
        context.bot.send_message(chat_id=update.effective_chat.id, text="Я не знаю, спроси у ChatGPT")
    else:
        # Handle non-text updates (optional)
        pass

def main() -> None:
    """Start the bot."""
    updater = Updater(TOKEN)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Handle all text messages
    message_handler = MessageHandler(Filters.text & (~Filters.command), reply_message)
    dp.add_handler(message_handler)

    # Start the Bot
    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()
